# Генерация датасета

В первой части проекта мы пришли к выводу, что для данной задачи лучше использовать дистанционный граф с параметром `d = 0.8`.\
Поэтому далее будем работать только с дистанционным графом.

In [222]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx

## Вспомогательные функции

Генерация случайных величин, которые имеют распределение $Exp(λ)$ и $LogNormal(0, σ)$

In [223]:
import math

lam0 = 1/(math.sqrt(math.e**2 - math.e))
sigma0 = 1

In [224]:
def generate_exp(n, lam=lam0):
  return np.random.exponential(scale=1 / lam, size=n)

def generate_lognormal(n, sigma=sigma0):
  return np.random.lognormal(mean=0, sigma=sigma, size=n)

Функция, которая строит дистанционный граф

In [225]:
# параметр дистанцонного графа
D = 0.8

In [226]:
def build_distance_graph(vertices):
    v = np.asarray(vertices)
    n = v.size

    G = nx.Graph()
    G.add_nodes_from(range(n))

    for i in range(n):
        for j in range(i+1, n):
            if abs(v[i] - v[j]) <= D:
                G.add_edge(i, j)

    return G

Создадим функции, которые будут считать характеристики дистанционного графа

In [227]:
import networkx as nx
from collections import Counter
from networkx.algorithms.approximation.dominating_set import min_weighted_dominating_set
from networkx.algorithms import maximal_independent_set

In [228]:
def chromatic_number(G: nx.Graph) -> int:
    """
    Функция для поиска хроматического числа графа G
    """
    coloring = nx.coloring.greedy_color(G, strategy='largest_first')
    return max(coloring.values(), default=-1) + 1

In [229]:
def clique_number(G: nx.Graph) -> int:
    """
    Функция для поиска кликового числа графа G
    """
    Gc = nx.complement(G)
    coloring = nx.coloring.greedy_color(Gc, strategy='largest_first')
    counts = Counter(coloring.values())
    return max(counts.values(), default=0)

In [230]:
def max_independent_set_size(G: nx.Graph) -> int:
    """
    Функция для поиска размера максимального 
    независимого множества графа G
    """
    mis = maximal_independent_set(G)
    return len(mis)

In [231]:
def domination_number(G: nx.Graph) -> int:
    """
    Функция для поиска числа 
    доминирования для графа G
    """
    D = min_weighted_dominating_set(G)
    return len(D)

In [232]:
def min_clique_cover_size(G: nx.Graph) -> int:
    """
    Функция для поиска размера максимального
    кликового покрытия графа G
    """
    Gc = nx.complement(G)
    coloring = nx.coloring.greedy_color(Gc, strategy='largest_first')
    return len(set(coloring.values()))

## Генерация датасета

In [233]:
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
import networkx as nx

In [234]:
SEED = 42
SAMPLES = 10000
N = [25, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

In [235]:

def generate_row(idx: int, seed: int) -> dict:
    rng = np.random.RandomState(seed + idx)
    distributions = {
        0: generate_exp,
        1: generate_lognormal
    }

    n = rng.choice(N)
    dist_label = int(rng.choice([0, 1]))
    gen_func = distributions[dist_label]
    
    vertices = gen_func(n)
    G = build_distance_graph(vertices)
    
    return {
        'n': n,
        'distribution': dist_label,
        'chromatic_number': chromatic_number(G),
        'clique_number': clique_number(G),
        'max_independent_set_size': max_independent_set_size(G),
        'domination_number': domination_number(G),
        'min_clique_cover_size': min_clique_cover_size(G)
    }

In [236]:
def generate_dataset(num_samples=SAMPLES, seed=SEED):
    records = Parallel(n_jobs=-1, verbose=0)(
        delayed(generate_row)(i, seed)
        for i in tqdm(range(num_samples), desc="Generating rows")
    )

    df = pd.DataFrame.from_records(records, columns=[
        'n', 'distribution',
        'chromatic_number', 'clique_number',
        'max_independent_set_size',
        'domination_number', 'min_clique_cover_size'
    ])
    df.to_csv('data/distance_graph_dataset.csv', index=False)

In [242]:
generate_dataset()

Generating rows: 100%|██████████| 10000/10000 [09:36<00:00, 17.35it/s]
